# **Global Biomes and Ecoregions Assessment**
<span style = "font-size: 1.5em;"> ***RESOLVE Biome and Ecoregion Dataset in 2017*** </span>

## Brief Description

### <span style = "font-size: 1em;"> ***Brief Description of The Asssessment*** </span>

The assessment will estimate the spatial distribution of ecoregions and biomes area in the region of interest  in 2017. 

<span style="font-size:1.1em;"> ***Input Data Needed*** </span>
- Shapefile
> - **roi_directory** : specify region of interest shapefiile
> - **ecoregions_directory** : select the global biomes and ecoregions shapefile

- Input Data Variables
> - **projectname (string)** : project name (example: "RVL_IDN_KubuRaya")
> - **projection**: UTM projection code in code (example: "EPSG:3857")
> - **roi_column_name** : variable name that specifiy unique ID for multipolygon analysis (example: "Name")

- Output directory folder
> - **output_directory** : select directory folder to put the output result

<span style="font-size:1.1em;"> ***Expected Output*** </span>
- Shapefile dataset
> - Biomess and Ecoregions Shapefile

- csv, graph and maps 
> - Biomes and Ecoregions distribution extent Area (Ha)

### <span style = "font-size: 1em;"> ***Brief Description of The Dataset*** </span>

The data shows global terrestrial ecoregions and biomes in 2017. Ecoregions are the ecosystem of regional extent which has a distinct assemblages of biodiversity at all taxa. The terrestrial ecoregions then are grouped into biomes where represent the forest and non-forest biomes. Biomes is a geographically extensive type of ecosystem which are characterized by the life forms of their dominant organisms, not necessarily by their particular species. On terrestrials, biomes are generally identified by their mature or older-growth vegetation whereas aquatic biome are distinguished by their dominant aquatic animals.

<span style="font-size:1.1em;"> ***Source*** </span>
- RESOLVE - Global Biomes and Ecoregions Extent Change 2017
Eric Dinerstein, David Olson, Anup Joshi, Carly Vynne, Neil D. Burgess, Eric Wikramanayake, Nathan Hahn, Suzanne Palminteri, Prashant Hedao, Reed Noss, Matt Hansen, Harvey Locke, Erle C Ellis, Benjamin Jones, Charles Victor Barber, Randy Hayes, Cyril Kormos, Vance Martin, Eileen Crist, Wes Sechrest, Lori Price, Jonathan E. M. Baillie, Don Weeden, Kierán Suckling, Crystal Davis, Nigel Sizer, Rebecca Moore, David Thau, Tanya Birch, Peter Potapov, Svetlana Turubanova, Alexandra Tyukavina, Nadia de Souza, Lilian Pintea, José C. Brito, Othman A. Llewellyn, Anthony G. Miller, Annette Patzelt, Shahina A. Ghazanfar, Jonathan Timberlake, Heinz Klöser, Yara Shennan-Farpón, Roeland Kindt, Jens-Peter Barnekow Lillesø, Paulo van Breugel, Lars Graudal, Maianna Voge, Khalaf F. Al-Shammari, Muhammad Saleem, An Ecoregion-Based Approach to Protecting Half the Terrestrial Realm, BioScience, Volume 67, Issue 6, June 2017, Pages 534–545, https://doi.org/10.1093/biosci/bix014


### <span style = "font-size: 1em;"> ***Note*** </span>

- The script will  estimate the **biomes and ecoregions extent area in 2020**

## Data Preparation

### Import modules and Relevant Datasets

In [1]:
%matplotlib inline
import os
import geopandas as gpd
import pandas as pd
from tkinter import Tk, filedialog, simpledialog, messagebox
from tkinter import * 
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.colors import ListedColormap, BoundaryNorm, LinearSegmentedColormap
from matplotlib import colors
from matplotlib.patches import Patch
import geoplot as gplt
import contextily as ctx
import plotly.express as px
import numpy as np
import geoplot.crs as gcrs
import mapclassify as mc
import cartopy.io.img_tiles as cimgt
import shapely.wkt

In [2]:
root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
roi_directory = filedialog.askopenfile(title = "Open RoI Shapefile") 
biomes_ecoregions_directory = filedialog.askopenfile(title = "Open Biomes and Ecoregions Shapefile")
# Output folder
output_directory = filedialog.askdirectory(title = "Specify Output Folder")

In [3]:
# output directory
out_dir = os.path.expanduser(output_directory+'/RapidScreening_BiomesEcoregionsAssessment')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

### Specify Input Data

In [5]:
# specify project name
projectname = simpledialog.askstring("Input", "Speficy project name",parent=root)

if projectname is not None:
    print("The script will assess ", projectname)
else:
    print("Name has not been submitted, Restart the code!")
    
#Specify Projection
projection = simpledialog.askstring("Input", "Specify Projection in UTM zone, example EPSG:3857", parent=root)

if projection is not None:
    print("All data will be reprojected to ", projection, ' canopy cover')
else:
    print("Projection has not specified, Restart the code!")
    
# column name for roi id
roi_column_name = simpledialog.askstring("Input", "Specify columns name in ROI attribute table to define boundary name",parent=root)

if roi_column_name is not None:
    print("Column name is ", roi_column_name)
else:
    print("Column has not identified, Restart the code!")

The script will assess  Jambi
All data will be reprojected to  EPSG:32748  canopy cover
Column name is  PROVINSI


## Spatial Distribution of Biomes and Ecoregions Extent

In [6]:
# roi shapefile
roi = gpd.read_file(roi_directory.name)
biomes_ecoregions = gpd.read_file(biomes_ecoregions_directory.name)

# reproject to UTM
roi_proj  = roi.to_crs(crs = projection)
biomes_ecoregions_proj = biomes_ecoregions.to_crs(crs = projection)

In [7]:
biomes_ecoregions_proj2 = biomes_ecoregions_proj.buffer(0)

C:\Users\FadliUwaisElqorni\.conda\envs\geemap\lib\site-packages\pygeos\constructive.py:175: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


In [27]:
biomes_ecoregions_proj["geometry"] = biomes_ecoregions_proj["geometry"].apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x, rounding_precision=4)))

In [14]:
# dissolve BIOMES
biomes_proj = biomes_ecoregions_proj[['BIOME_NUM','BIOME_NAME','COLOR_BIO','geometry']].dissolve(by = ['BIOME_NUM','BIOME_NAME','COLOR_BIO'])

GEOSException: TopologyException: side location conflict at -2392450.9566475092 29955059.337994698. This can occur if the input geometry is invalid.

In [8]:
# intersect roi with biomes and ecoregions
biomes_ecoregions_roi_proj = gpd.overlay(roi_proj, biomes_ecoregions_proj, how = "intersection", keep_geom_type=False)

C:\Users\FadliUwaisElqorni\.conda\envs\geemap\lib\site-packages\pygeos\constructive.py:175: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


In [ ]:
# calculate area
# calculate area
biomes_ecoregions_roi_proj["Area_Ha"] = biomes_ecoregions_roi_proj['geometry'].area/10000

### Data Visualization

In [ ]:
roi_proj_4326 = roi_proj.to_crs(crs = 'EPSG:4326')
biomes_ecoregions_roi_proj_4326 = biomes_ecoregions_roi_proj.to_crs(crs = 'EPSG:4326')

In [ ]:
colors_ecoregions = biomes_ecoregions_roi_proj_4326['COLOR'].tolist()
colormap_ecoregions = LinearSegmentedColormap.from_list([1, 2],colors_mangrove)

colors_biomes = biomes_ecoregions_roi_proj_4326['COLOR_BIO'].tolist()


scheme_mnm = mc.UserDefined(mangrove_nonmangrove_roi_proj_4326['Data'], bins = [1, 2])